In [ ]:
def evaluation_protocol(self, batch):
        edge_list, labels = batch
        batch_size = edge_list.shape[0]

        # combine heads, tails and labels
        triplets = torch.hstack([edge_list, labels.reshape(-1,1)])

        # repeat all triplets for n_entities times
        triplets = triplets[:,np.newaxis,:].repeat(1,self.num_entities,1)

        true_pos_total = list()
        rank_pos_list = list()

        # repeat corruption for both head and tail
        for pos in [0,1]:
            x = triplets.detach().clone()
            
            # replace all heads with list of all possible entities
            x[:,:,pos] = torch.tensor(range(self.num_entities))[np.newaxis,:].repeat(batch_size,1).to(self.device)

            # triplets are arranged as (head, tail, label)
            head = self.entity_mat.weight[x[:,:,0]]
            tail = self.entity_mat.weight[x[:,:,1]]
            rel = self.relation_mat.weight[x[:,:,2]]

            print(x[:,:,1])

            # compute triplet score
            norms = torch.norm(head + rel - tail, dim=2, p=self.p_norm)
            print(norms[0,:5])
            
            # get index positions of sorted norms for each triplet
            rankings = torch.vstack([torch.argsort(x) for x in norms.unbind(dim=0)])
            print(rankings[0,:5])

            #print(torch.vstack([torch.sort(x)[0] for x in norms.unbind(dim=0)])[0])

            # find position of heads within the rankings
            rank_pos = torch.where(rankings == edge_list[:,pos].reshape(-1,1))[1]

            rank_pos_list.append(rank_pos)
            true_pos_total.append(rank_pos < 10)

        mean_rank = torch.vstack(rank_pos_list)#.float().mean()
        hits_at_10 = torch.vstack(true_pos_total)#.float().mean()*100

        return mean_rank, hits_at_10

In [ ]:
x = next(iter(dm.train_dataloader()))
_ = evaluation_protocol(model, x)

edge = (2, 11329, 1)
head = model.entity_mat.weight[edge[0]]
tail = model.entity_mat.weight[edge[1]]
label = model.relation_mat.weight[edge[2]]
torch.norm(head + label - tail, p=model.p_norm, dim=-1)